## Imports

In [32]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df.columns = df.columns.str.replace(' ', '_')
df_test.columns = df_test.columns.str.replace(' ', '_')


## Pipelines

In [33]:
impute_onehot = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
impute_onehot.set_output(transform="pandas")
impute_onehot_cols = ['Style', 'Color', 'Brand', 'Material']

size_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Medium')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
]) 
size_pipeline.set_output(transform="pandas")
size_pipeline_cols = ['Size']

yes_no_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
]) 
yes_no_pipeline.set_output(transform="pandas")
yes_no_pipeline_cols = ['Waterproof', 'Laptop_Compartment']

weight_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=18.0)),
    ('scaler', StandardScaler())
])
weight_pipeline.set_output(transform="pandas")
weight_pipeline_cols = ['Weight_Capacity_(kg)']

compartments_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler())
])
compartments_pipeline.set_output(transform='pandas')
compartments_pipeline_cols = ['Compartments']


## Column Transformer

In [41]:
preprocessor = ColumnTransformer([
    ('style_color_brand_material', impute_onehot, impute_onehot_cols),
    ('size', size_pipeline, size_pipeline_cols),
    ('waterproof_laptopcompartment', yes_no_pipeline, yes_no_pipeline_cols),
    ('weight_pipeline', weight_pipeline, weight_pipeline_cols),
    ('compartments', compartments_pipeline, compartments_pipeline_cols)
])
preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('style_color_brand_material',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Unknown',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Style', 'Color', 'Brand', 'Material']),
                                ('size',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Medium',
                                                                strategy='constant')),
                                                 ('o...
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1))]),
                                 ['Waterproof', 'Laptop_Compartment']),
                                ('weight_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=18.0,
                                                                strategy='constant')),
                                                 ('scaler', StandardScaler())]),
                                 ['Weight_Capacity_(kg)']),
                                ('compartments',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Compartments'])])

## Split

In [69]:
from sklearn.model_selection import train_test_split

id_col = 'id'
target_col = 'Price'
y = df[target_col]
X = df.drop([id_col,target_col],axis=1)

X_transformed = preprocessor.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.2, random_state=13)

# Simple models
### Wybór modeli:
- XGBRegressor
- LightGBM
- XGBoost
- CatBoost
- KNN Imputer (?)


# Prepare Test Data

In [77]:
test_id_col=df_test.id
X_test= df_test.drop('id',axis=1)
X_test_transformed = preprocessor.fit_transform(X_test)

# XGBRegressor

In [78]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=50, max_depth=4, learning_rate=0.05)
xgb.fit(X_train, y_train)

test_predicted = xgb.predict(X_test_transformed)
submission = pd.DataFrame({
    'id': test_id_col,
    'Price': test_predicted
})
submission.to_csv('submission1.csv', index=False)

# XGBRegressor + Optuna (Basic)

In [ ]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import root_mean_squared_error

def objective(trial):
    n_estimators=trial.suggest_int('n_estimators',100,500)
    max_depth=trial.suggest_int('max_depth',3,10)
    learning_rate=trial.suggest_float('learning_rate', 0.01,0.07)
    model = XGBRegressor(n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate)
    score = cross_val_score(model, X_train,y_train, cv=5, scoring='neg_root_mean_squared_error')
    return score.mean()

study =optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, show_progress_bar=True )

# [I 2025-02-07 13:51:51,563] Trial 0 finished with value: -39.03383156751836 and parameters: {'n_estimators': 153, 'max_depth': 3, 'learning_rate': 0.06991325421006729}. Best is trial 0 with value: -39.03383156751836.



In [84]:
xgb = XGBRegressor(n_estimators=153, max_depth=3, learning_rate=0.0699)
xgb.fit(X_train, y_train)
test_predicted = xgb.predict(X_test_transformed)
submission = pd.DataFrame({
    'id': test_id_col,
    'Price': test_predicted
})
submission.to_csv('submission2.csv', index=False)